In [1]:
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# 读取灌溉数据
file_path = 'IWU.ens.Y2018.nc'
ds = xr.open_dataset(file_path, engine='netcdf4')

result_df = pd.DataFrame()
for month in range(12):
    
    # 获取 2018 年 k 月月份截面（根据索引映射到实际月份）
    subset_data = ds['IWUens'][month, :, :]
    
    # 获取非 nan 值的位置索引
    non_nan_lon, non_nan_lat = np.where(subset_data.values >= 0)
    
    # 获取非 nan 且非 0 值的实际数据（若需过滤 0，可加 (subset_data.values != 0) 条件）
    non_nan_values = subset_data.values[non_nan_lon, non_nan_lat]
    
    # 转换为真实经纬度值
    real_lat = subset_data.lat.values[non_nan_lat] #纬度
    real_lon = subset_data.lon.values[non_nan_lon] #经度

    month_df = pd.DataFrame({
            'lat': real_lat,
            'lon': real_lon,
            f'{1+month}月': np.round(non_nan_values,2)  # 用月份区分不同列
        })
        
    # 合并数据：首次循环直接赋值，后续按经纬度关联合并
    if result_df.empty:
        result_df = month_df
    else:
        # 按经纬度匹配合并，确保同一经纬度的月份数据对应
        result_df = result_df.merge(month_df, on=['lat', 'lon'], how='outer')

# 合并经纬度后的缺失的灌溉值用0补充
result_df = result_df.fillna(0)   
result_df.to_excel('./2018年灌溉用水量(IWU).xlsx', index=False, sheet_name='2018年灌溉用水量（IWU）') 

ds.close()